In [1]:
import keras
import numpy as np
import scipy.io
from pathlib import Path
import os

Using TensorFlow backend.


## Open labels

In [2]:
path = 'C:\\Users\\Rodolfo\\Documents\\algoritmos-git\\custom_cnn\\Data\\Processados\\'

In [3]:
y_train = np.load(Path(path,'y_train.npy'))
y_test = np.load(Path(path, 'y_test.npy'))
y_train = np.expand_dims(y_train, axis=1)
y_test = np.expand_dims(y_test, axis=1)

In [4]:
from keras.utils import np_utils
# one-hot encode the labels
num_classes = len(np.unique(y_train))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
y_train

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [6]:
y_test

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

# Open data

In [7]:
path = Path(path, '28_28')
X_train_1 = np.load(Path(path, 'X_train_1.npy'))
X_train_2 = np.load(Path(path, 'X_train_2.npy'))
X_test_1 = np.load(Path(path, 'X_test_1.npy'))
X_test_2 = np.load(Path(path, 'X_test_2.npy'))
print(X_train_1.shape)
print(X_train_2.shape)
print(X_test_1.shape)
print(X_test_2.shape)

(2200, 28, 28)
(2200, 28, 28)
(1000, 28, 28)
(1000, 28, 28)


### Shape desejado: x_train shape: (45000, 32, 32, 3) -> n_obs, n_row, n_col, n_chan

In [8]:
n_obs, n_row, n_col = X_train_1.shape
n_chan=2
X_train = np.zeros((n_obs, n_row, n_col, n_chan))
X_train.shape

(2200, 28, 28, 2)

In [9]:
X_train[:,:,:,0]=X_train_1
X_train[:,:,:,1]=X_train_2
X_train.shape

(2200, 28, 28, 2)

In [10]:
n_obs, n_row, n_col = X_test_1.shape
n_chan=2
X_test = np.zeros((n_obs, n_row, n_col, n_chan))
X_test.shape

(1000, 28, 28, 2)

In [11]:
X_test[:,:,:,0]=X_test_1
X_test[:,:,:,1]=X_test_2
X_test.shape

(1000, 28, 28, 2)

# Shuffle X_train

In [12]:
arr = np.arange(y_train.shape[0])
np.random.shuffle(arr)
X_train = X_train[arr,:,:,:]
y_train = y_train[arr]

### 3. Rescale the Images by Dividing Every Pixel in Every Image by 255

In [13]:
# rescale [0,255] --> [0,1]
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

### 4.  Break Dataset into Training, Testing, and Validation Sets

In [14]:
# break training set into training and validation sets
(X_train, X_valid) = X_train[:1800], X_train[1800:]
(y_train, y_valid) = y_train[:1800], y_train[1800:]

# print shape of training set
print('x_train shape:', X_train.shape)

# print number of training, validation, and test images
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
print(X_valid.shape[0], 'validation samples')

x_train shape: (1800, 28, 28, 2)
1800 train samples
1000 test samples
400 validation samples


### 5. Define the Model Architecture 

In [18]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', strides=(1,1), input_shape=(28, 28, 2)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu', strides=(2,2)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', strides=(1,1)))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 16)        144       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 32)          2080      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 3, 3, 32)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          8256      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 1, 1, 64)          0         
__________

### 6. Compile the Model 

In [19]:
# compile the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
                  metrics=['accuracy'])

### 7. Train the Model 

In [25]:
from keras.callbacks import ModelCheckpoint   
model_path = 'model.weights.best.hdf5'
# train the model
checkpointer = ModelCheckpoint(filepath=model_path, verbose=1, 
                               save_best_only=True)
hist = model.fit(X_train, y_train, batch_size=32, epochs=500,
          validation_data=(X_valid, y_valid), callbacks=[checkpointer], 
          verbose=2, shuffle=True)

Train on 1800 samples, validate on 400 samples
Epoch 1/500
 - 2s - loss: 0.4039 - acc: 0.8094 - val_loss: 0.9117 - val_acc: 0.6150

Epoch 00001: val_loss improved from inf to 0.91167, saving model to model.weights.best.hdf5
Epoch 2/500
 - 1s - loss: 0.3893 - acc: 0.8244 - val_loss: 0.7286 - val_acc: 0.6750

Epoch 00002: val_loss improved from 0.91167 to 0.72861, saving model to model.weights.best.hdf5
Epoch 3/500
 - 1s - loss: 0.4109 - acc: 0.8117 - val_loss: 0.6867 - val_acc: 0.6750

Epoch 00003: val_loss improved from 0.72861 to 0.68674, saving model to model.weights.best.hdf5
Epoch 4/500
 - 1s - loss: 0.3930 - acc: 0.8200 - val_loss: 0.6541 - val_acc: 0.6700

Epoch 00004: val_loss improved from 0.68674 to 0.65413, saving model to model.weights.best.hdf5
Epoch 5/500
 - 1s - loss: 0.3750 - acc: 0.8322 - val_loss: 0.7303 - val_acc: 0.6575

Epoch 00005: val_loss did not improve from 0.65413
Epoch 6/500
 - 1s - loss: 0.3504 - acc: 0.8583 - val_loss: 0.8313 - val_acc: 0.6800

Epoch 00006:


Epoch 00059: val_loss did not improve from 0.65413
Epoch 60/500
 - 1s - loss: 0.1429 - acc: 0.9428 - val_loss: 1.5441 - val_acc: 0.6000

Epoch 00060: val_loss did not improve from 0.65413
Epoch 61/500
 - 1s - loss: 0.1295 - acc: 0.9517 - val_loss: 1.3906 - val_acc: 0.6600

Epoch 00061: val_loss did not improve from 0.65413
Epoch 62/500
 - 1s - loss: 0.1550 - acc: 0.9450 - val_loss: 1.2244 - val_acc: 0.6850

Epoch 00062: val_loss did not improve from 0.65413
Epoch 63/500
 - 1s - loss: 0.1138 - acc: 0.9550 - val_loss: 1.5390 - val_acc: 0.6575

Epoch 00063: val_loss did not improve from 0.65413
Epoch 64/500
 - 1s - loss: 0.1358 - acc: 0.9406 - val_loss: 1.3598 - val_acc: 0.6800

Epoch 00064: val_loss did not improve from 0.65413
Epoch 65/500
 - 1s - loss: 0.1220 - acc: 0.9517 - val_loss: 1.7519 - val_acc: 0.6375

Epoch 00065: val_loss did not improve from 0.65413
Epoch 66/500
 - 1s - loss: 0.1299 - acc: 0.9489 - val_loss: 1.4416 - val_acc: 0.6775

Epoch 00066: val_loss did not improve fr


Epoch 00119: val_loss did not improve from 0.65413
Epoch 120/500
 - 1s - loss: 0.0751 - acc: 0.9733 - val_loss: 1.9497 - val_acc: 0.6750

Epoch 00120: val_loss did not improve from 0.65413
Epoch 121/500
 - 1s - loss: 0.0789 - acc: 0.9700 - val_loss: 1.6819 - val_acc: 0.6625

Epoch 00121: val_loss did not improve from 0.65413
Epoch 122/500
 - 1s - loss: 0.0796 - acc: 0.9689 - val_loss: 1.9437 - val_acc: 0.6875

Epoch 00122: val_loss did not improve from 0.65413
Epoch 123/500
 - 1s - loss: 0.1023 - acc: 0.9644 - val_loss: 1.7826 - val_acc: 0.6675

Epoch 00123: val_loss did not improve from 0.65413
Epoch 124/500
 - 1s - loss: 0.0834 - acc: 0.9689 - val_loss: 2.2680 - val_acc: 0.6400

Epoch 00124: val_loss did not improve from 0.65413
Epoch 125/500
 - 1s - loss: 0.0881 - acc: 0.9650 - val_loss: 1.8942 - val_acc: 0.6650

Epoch 00125: val_loss did not improve from 0.65413
Epoch 126/500
 - 1s - loss: 0.0954 - acc: 0.9656 - val_loss: 2.1836 - val_acc: 0.6600

Epoch 00126: val_loss did not imp

 - 1s - loss: 0.0902 - acc: 0.9639 - val_loss: 2.1271 - val_acc: 0.6400

Epoch 00179: val_loss did not improve from 0.65413
Epoch 180/500
 - 1s - loss: 0.0439 - acc: 0.9844 - val_loss: 2.2760 - val_acc: 0.6575

Epoch 00180: val_loss did not improve from 0.65413
Epoch 181/500
 - 2s - loss: 0.0859 - acc: 0.9728 - val_loss: 2.1105 - val_acc: 0.6825

Epoch 00181: val_loss did not improve from 0.65413
Epoch 182/500
 - 1s - loss: 0.0801 - acc: 0.9683 - val_loss: 2.2403 - val_acc: 0.6500

Epoch 00182: val_loss did not improve from 0.65413
Epoch 183/500
 - 2s - loss: 0.0751 - acc: 0.9750 - val_loss: 2.0612 - val_acc: 0.6675

Epoch 00183: val_loss did not improve from 0.65413
Epoch 184/500
 - 2s - loss: 0.0537 - acc: 0.9800 - val_loss: 2.5243 - val_acc: 0.6400

Epoch 00184: val_loss did not improve from 0.65413
Epoch 185/500
 - 1s - loss: 0.0904 - acc: 0.9733 - val_loss: 1.8905 - val_acc: 0.6850

Epoch 00185: val_loss did not improve from 0.65413
Epoch 186/500
 - 1s - loss: 0.0939 - acc: 0.9744


Epoch 00238: val_loss did not improve from 0.65413
Epoch 239/500
 - 1s - loss: 0.0787 - acc: 0.9756 - val_loss: 2.1037 - val_acc: 0.6575

Epoch 00239: val_loss did not improve from 0.65413
Epoch 240/500
 - 1s - loss: 0.0633 - acc: 0.9800 - val_loss: 2.3220 - val_acc: 0.6450

Epoch 00240: val_loss did not improve from 0.65413
Epoch 241/500
 - 1s - loss: 0.0610 - acc: 0.9817 - val_loss: 2.4328 - val_acc: 0.6425

Epoch 00241: val_loss did not improve from 0.65413
Epoch 242/500
 - 1s - loss: 0.0625 - acc: 0.9828 - val_loss: 2.7189 - val_acc: 0.6450

Epoch 00242: val_loss did not improve from 0.65413
Epoch 243/500
 - 1s - loss: 0.0727 - acc: 0.9761 - val_loss: 2.3909 - val_acc: 0.6675

Epoch 00243: val_loss did not improve from 0.65413
Epoch 244/500
 - 2s - loss: 0.0745 - acc: 0.9722 - val_loss: 2.0107 - val_acc: 0.6500

Epoch 00244: val_loss did not improve from 0.65413
Epoch 245/500
 - 2s - loss: 0.0839 - acc: 0.9767 - val_loss: 1.9296 - val_acc: 0.6800

Epoch 00245: val_loss did not imp

 - 1s - loss: 0.0858 - acc: 0.9767 - val_loss: 2.5504 - val_acc: 0.6325

Epoch 00298: val_loss did not improve from 0.65413
Epoch 299/500
 - 1s - loss: 0.0748 - acc: 0.9778 - val_loss: 2.5085 - val_acc: 0.6500

Epoch 00299: val_loss did not improve from 0.65413
Epoch 300/500
 - 1s - loss: 0.0890 - acc: 0.9711 - val_loss: 2.4760 - val_acc: 0.6250

Epoch 00300: val_loss did not improve from 0.65413
Epoch 301/500
 - 1s - loss: 0.0598 - acc: 0.9817 - val_loss: 2.7175 - val_acc: 0.6500

Epoch 00301: val_loss did not improve from 0.65413
Epoch 302/500
 - 1s - loss: 0.0648 - acc: 0.9778 - val_loss: 2.3547 - val_acc: 0.6375

Epoch 00302: val_loss did not improve from 0.65413
Epoch 303/500
 - 1s - loss: 0.0627 - acc: 0.9800 - val_loss: 2.3886 - val_acc: 0.6600

Epoch 00303: val_loss did not improve from 0.65413
Epoch 304/500
 - 1s - loss: 0.0714 - acc: 0.9822 - val_loss: 2.6353 - val_acc: 0.6575

Epoch 00304: val_loss did not improve from 0.65413
Epoch 305/500
 - 1s - loss: 0.0636 - acc: 0.9783


Epoch 00357: val_loss did not improve from 0.65413
Epoch 358/500
 - 2s - loss: 0.0891 - acc: 0.9722 - val_loss: 2.1612 - val_acc: 0.6450

Epoch 00358: val_loss did not improve from 0.65413
Epoch 359/500
 - 1s - loss: 0.0592 - acc: 0.9833 - val_loss: 2.2612 - val_acc: 0.6425

Epoch 00359: val_loss did not improve from 0.65413
Epoch 360/500
 - 1s - loss: 0.0567 - acc: 0.9839 - val_loss: 2.3665 - val_acc: 0.6325

Epoch 00360: val_loss did not improve from 0.65413
Epoch 361/500
 - 1s - loss: 0.0893 - acc: 0.9767 - val_loss: 2.1494 - val_acc: 0.6525

Epoch 00361: val_loss did not improve from 0.65413
Epoch 362/500
 - 1s - loss: 0.0663 - acc: 0.9733 - val_loss: 2.3043 - val_acc: 0.6475

Epoch 00362: val_loss did not improve from 0.65413
Epoch 363/500
 - 1s - loss: 0.0680 - acc: 0.9767 - val_loss: 2.1480 - val_acc: 0.6600

Epoch 00363: val_loss did not improve from 0.65413
Epoch 364/500
 - 2s - loss: 0.0647 - acc: 0.9844 - val_loss: 2.1844 - val_acc: 0.6600

Epoch 00364: val_loss did not imp

 - 2s - loss: 0.0640 - acc: 0.9794 - val_loss: 2.3804 - val_acc: 0.6175

Epoch 00417: val_loss did not improve from 0.65413
Epoch 418/500
 - 2s - loss: 0.0678 - acc: 0.9789 - val_loss: 2.6376 - val_acc: 0.6350

Epoch 00418: val_loss did not improve from 0.65413
Epoch 419/500
 - 2s - loss: 0.0432 - acc: 0.9839 - val_loss: 2.8320 - val_acc: 0.6600

Epoch 00419: val_loss did not improve from 0.65413
Epoch 420/500
 - 2s - loss: 0.0886 - acc: 0.9750 - val_loss: 2.2806 - val_acc: 0.6500

Epoch 00420: val_loss did not improve from 0.65413
Epoch 421/500
 - 2s - loss: 0.0654 - acc: 0.9811 - val_loss: 2.8292 - val_acc: 0.6650

Epoch 00421: val_loss did not improve from 0.65413
Epoch 422/500
 - 2s - loss: 0.0975 - acc: 0.9706 - val_loss: 3.0447 - val_acc: 0.6450

Epoch 00422: val_loss did not improve from 0.65413
Epoch 423/500
 - 2s - loss: 0.0773 - acc: 0.9789 - val_loss: 2.3855 - val_acc: 0.6675

Epoch 00423: val_loss did not improve from 0.65413
Epoch 424/500
 - 2s - loss: 0.0536 - acc: 0.9817


Epoch 00476: val_loss did not improve from 0.65413
Epoch 477/500
 - 2s - loss: 0.0654 - acc: 0.9806 - val_loss: 2.3148 - val_acc: 0.6600

Epoch 00477: val_loss did not improve from 0.65413
Epoch 478/500
 - 2s - loss: 0.0516 - acc: 0.9850 - val_loss: 2.4534 - val_acc: 0.6325

Epoch 00478: val_loss did not improve from 0.65413
Epoch 479/500
 - 2s - loss: 0.0768 - acc: 0.9756 - val_loss: 2.4277 - val_acc: 0.6525

Epoch 00479: val_loss did not improve from 0.65413
Epoch 480/500
 - 2s - loss: 0.0556 - acc: 0.9839 - val_loss: 2.4838 - val_acc: 0.6300

Epoch 00480: val_loss did not improve from 0.65413
Epoch 481/500
 - 2s - loss: 0.0727 - acc: 0.9728 - val_loss: 2.0817 - val_acc: 0.6200

Epoch 00481: val_loss did not improve from 0.65413
Epoch 482/500
 - 2s - loss: 0.0603 - acc: 0.9844 - val_loss: 2.5142 - val_acc: 0.6425

Epoch 00482: val_loss did not improve from 0.65413
Epoch 483/500
 - 2s - loss: 0.0549 - acc: 0.9800 - val_loss: 2.6342 - val_acc: 0.6500

Epoch 00483: val_loss did not imp

### 8. Load the Model with the Best Validation Accuracy

In [27]:
# load the weights that yielded the best validation accuracy
model.load_weights(model_path)

### 9. Calculate Classification Accuracy on Test Set

In [28]:
# evaluate and print test accuracy
score = model.evaluate(X_test, y_test, verbose=0)
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.688
